## Firecloud: Uploading Metadata to Firecloud
Mimoun Cadosch 7/17

In [78]:
from firecloud import api as firecloud_api
import pandas as pd
import os, sys
import datetime
from IPython.display import display

In [74]:
## Resources
# https://github.com/broadinstitute/fiss/blob/master/firecloud/api.py
# https://github.com/broadinstitute/firecloud-tools

In [75]:
# res = firecloud_api.delete_workspace_config(namespace, workspace, "tsca", "Mutect2TumorOnly")
# res = firecloud_api.delete_workspace_config(namespace, workspace, 'tsca', 'CreatePanelOfNormalsGATK')
# delete_sample(namespace, workspace, "AA66-Tumor-SM-F29RQ")
# res = delete_sample_set(namespace, workspace, sample_set_id='CumPoN_1920')
# firecloud_api.delete_repository_method('tsca', 'CallSomaticCNV', 10)

In [76]:
namespace = "nci-mimoun-bi-org"
workspace = "CCLF_TSCA"
google_bucket_id = "fc-35446f22-ea37-483a-bd6c-5e9fc56851ff"
path_to_all_samples_info = "paths_to_samples_info.xlsx"
# batches_info = pd.read_excel('paths_to_samples_info.xlsx')
# Must be True
filesystem_mounted = os.path.ismount('/xchip/clf/')

In [41]:
def delete_sample(namespace, workspace, sample_id):
    """Delete sample from workspace/namespace
    Args: 
        Self-explanatory
    Returns: 
        HTTP Response
    """
    body = [{"entityType": "sample", "entityName": sample_id}]
    res = firecloud_api.delete_entities(namespace, workspace, body)
    return res

In [42]:
def delete_sample_set(namespace, workspace, sample_set_id):
    """Delete sample set from workspace/namespace
    Args: 
        Self-explanatory
    Returns: 
        HTTP Response
    """
    body = [{"entityType": "sample_set", "entityName": sample_set_id}]
    res = firecloud_api.delete_entities(namespace, workspace, body)
    return res

In [43]:
def delete_workspace_config(namespace, workspace, cnamespace, config):
    """Delete workspace configuration
    Args: 
        Self-explanatory
    Returns: 
        HTTP Response
    """
    res = firecloud_api.delete_workspace_config(namespace, workspace, cnamespace, config)
    return res

In [44]:
def delete_entity_attributes(namespace, workspace, entity_type, entity_name, attrs):
    """Delete entity attributes
    Args: 
    - updates: list of attributes to delete
    """
    attr_update = [{"op": "RemoveAttribute", "attributeName":  attr} for attr in attrs]
    res = firecloud_api.update_entity(namespace, workspace, entity_type, entity_name, attr_update)
    return res

In [45]:
def upload_entities_from_tsv(namespace, workspace, entities_tsv_file):
    """Upload entities from tsv file
    Args: 
        Self-explanatory
        entities_tsv_file: path to tsv file
    Returns: 
        HTTP Response
    """
    res = firecloud_api.upload_entities_tsv(namespace, workspace, entities_tsv=entities_tsv_file)
    return res

In [46]:
def prepare_patients_for_metadata_import(path, tsca_id):
    """Create participant entities file for Firecloud. 
    Patients need to exist before you can upload their respective samples
    Args:
        path_id: path to file ending in {}.import_samples.txt
        tsca_id: tsca id
    Saves: 
        ./tsca_id/fc_upload_patients_tsca_{tsca_id}.csv:
            contains patient ids in tsca batch
    """    
    raw = pd.read_table(path)
    print( "%d Participants in this batch" % raw['individual_id'].unique().shape[0] )
    # Data to upload
    data = pd.DataFrame(raw.individual_id.drop_duplicates()).rename(columns={'individual_id':'entity:participant_id'})
    os.system('mkdir -p %s'%tsca_id)
    filename = './%s/fc_upload_patients_tsca_%s.txt' % (tsca_id, tsca_id)
    data.to_csv(filename, '\t', index=False)

In [47]:
def prepare_batch_sample_set_for_metadata_import(path, tsca_id):
    """Create sample_set entities file for Firecloud.
    A sample for a given batch 
    Args:
        path: path to file ending in {}.import_samples.txt
        tsca_id: batch tsca id
    """
    raw = pd.read_table(path)
    print( "%d Samples in this batch" % raw.shape[0] )

    # Data to upload
    data = pd.concat([pd.DataFrame(index=raw.index, columns=['membership:sample_set_id'], data=tsca_id), \
                      raw.sample_id], axis=1)
    os.system('mkdir -p %s'%tsca_id)
    filename = './%s/fc_upload_sample_set_tsca_%s.txt' % (tsca_id, tsca_id)
    data.to_csv(filename, '\t', index=False)

In [48]:
def prepare_batch_samples_for_metadata_import(path, tsca_id, google_bucket_id):
    """Prepare the file to import samples metadata to firecloud
    Args:
        path_id: path to file ending in {}.import_samples.txt
        tsca_id: TSCAXX
        google_bucket_id: id of google bucket ('gs://google_bucket_id')
    Returns:
        pd.DF of data ready for import
    Saves:
        ./{tsca_id}/fc_upload_samples_tsca_{tsca_id}.txt
    """
    # Import raw data
    data = pd.read_table(path)
    
    # Rename columns to match firecloud requirements
    data = data.rename(columns={'sample_id':'entity:sample_id', 'individual_id':'participant_id'})
    
    # Locations of BAM files in google bucket
    path_in_bucket_full = "gs://%s/seq_data/%s" % (google_bucket_id, tsca_id)

    # Extract bam filename
    data['bam_filename'] = data.apply(lambda row: row['clean_bam_file_capture'].split('/')[-1], axis=1)
    
    # Create bai filename (change extension on .bam file)
    data['bai_filename'] = data.apply(lambda row: "%s%s" %(row['bam_filename'][:-3], 'bai'), axis=1)
    
    # Change BAM path from xchip to Google cloud
    data['clean_bam_file_capture'] = \
        data.apply( lambda row: "%s/%s/%s" \
                   %(path_in_bucket_full, row['external_id_validation'], row['bam_filename']), axis=1)
    
    # Add location of .bai file 
    data['clean_bai_file_capture'] = \
        data.apply( lambda row: "%s/%s/%s" \
                   %(path_in_bucket_full, row['external_id_validation'], row['bai_filename']), axis=1)
       
    # Add TSCA ID
    data['tsca_id'] = tsca_id
        
    return data

In [49]:
def prepare_panel_of_normals_for_metadata_import(paths, N, name):
    """Create panel of normals sample set for Firecloud from multiple TSCA batches
    Args:
        paths: (list) paths to file ending in {}.import_samples.txt
        N: (int) number of samples in panel of normals
        name: (string) name of Panel of Normals
    """    
    dfs = [ pd.read_table(paths[0]) ]
    for i, path in enumerate(paths[1:]):
        df_to_concat = pd.read_table(path)
        dfs.append(df_to_concat)
    df = pd.concat(dfs, axis=0)
    # Shuffle samples to pick from all batches
    df = df.sample(frac=1).reset_index(drop=True)
    normals = df[df.sample_type=="Normal"][:N]['sample_id']
    if N==-1: print ("Creating panel of %d normals" %normals.shape[0])
    else: print ("Creating panel of %d normals" %N)
    
    data = pd.concat([pd.DataFrame(index=normals.index, columns=['membership:sample_set_id'], data=name), \
                        normals], axis=1)

    os.system('mkdir -p PoNs')
    filename = './PoNs/fc_upload_PoN_%s.txt' % (name)
    data.to_csv(filename, '\t', index=False)

In [50]:
def write_df_to_csv(data, tsca_id):
    data.to_csv('%s/fc_upload_samples_tsca_%s.txt' % (tsca_id, tsca_id), sep='\t', index=False)

In [51]:
def get_samples_multiple_batches(paths_to_samples_info, google_bucket_id, sublist=None):
    """Compile samples from multiple batches
    Args: Self-explanatory
        - paths_to_samples_info: .xlsx file containing paths to files containing sample_info
        - sublist: list of tsca_ids to only compile data from certain batches. If None, compile data from all batches.
    Returns: 
        - df with samples from all batches
    """
    paths_to_samples_info = pd.read_excel(paths_to_samples_info, index_col=0)
    df_list = []

    for tsca_id, paths in paths_to_samples_info.iterrows():
        if sublist is not None and tsca_id not in sublist:
            continue
        # Make data Firecloud-compatible
        batch_data = prepare_batch_samples_for_metadata_import(paths.path_to_samples_info, tsca_id, google_bucket_id)
        df_list.append(batch_data)

    all_samples = pd.concat(df_list, axis=0)
    return all_samples

In [52]:
def add_matching_samples(all_samples, batch_samples):
    """Add sample_id and bam filepath of matching normals and primary tumor tissue for every sample
    in batch_samples
    Args:
        - all_samples: df with target samples we want to find matches in
        - batch_samples: df with source samples we want to find matches for
    Returns: 
        - batch_samples (augmented)
    """
    for index, row in batch_samples.iterrows():
        # Find all samples from same individual (same individual_id, different sample_id)
        patient_samples = all_samples[ (all_samples['participant_id'] == row['participant_id']) \
                                      & (all_samples['entity:sample_id'] != row['entity:sample_id']) ]

        # NOTE: If more than one match tumor tissue or match normal found, select one at random.
        # The match normal is used to compute allelic fractions in Mutect2, so for now we ignore the conditions it was grown in.

        # Tumor tissue: Add primary tumor tissue
        match_primary_tumor = patient_samples[ patient_samples['external_id_validation'] \
                                              .str.contains('primary|prim|tissue|tiss') ]
        #    > No primary tumor tissue found
        if match_primary_tumor.empty:
            batch_samples.loc[index, 'match_primary_tumor_sample_id'] = "NA"
            batch_samples.loc[index, 'match_primary_tumor_bam_file'] = "NA"
        #    > Tumor tissue found
        elif match_primary_tumor.shape[0] > 0:
            match_primary_tumor = match_primary_tumor.sample(n=1)
            batch_samples.loc[index, 'match_primary_tumor_sample_id'] = match_primary_tumor['entity:sample_id'].item()
            batch_samples.loc[index, 'match_primary_tumor_bam_file'] = match_primary_tumor['clean_bam_file_capture'].item()

        # Add match normal
        match_normal = patient_samples[ patient_samples['sample_type'] == "Normal"]
        #   > No match normal found
        if match_normal.empty: 
            batch_samples.loc[index, 'match_normal_sample_id'] = "NA"
            batch_samples.loc[index, 'match_normal_bam_file'] = "NA"
        #   > Match normal found
        elif match_normal.shape[0] > 0:
            match_normal = match_normal.sample(n=1)
            batch_samples.loc[index, 'match_normal_sample_id'] = match_normal['entity:sample_id'].item()
            batch_samples.loc[index, 'match_normal_bam_file'] = match_normal['clean_bam_file_capture'].item()
            
    return batch_samples

In [53]:
def prepare_batch_data_for_metadata_import(tsca_id, path_to_batch_samples_info, path_to_all_samples_info):    
    """Prepare batch metadata for uploading to Firecloud
    Args:
        - path_to_batch_samples_info: path to info on batch samples
        - path_to_all_samples_info: path to info on all samples
    """
    prepare_patients_for_metadata_import(path_to_batch_samples_info, tsca_id)
    prepare_batch_sample_set_for_metadata_import(path_to_batch_samples_info, tsca_id)
    prepare_batch_samples = batch_samples_for_metadata_import(path_to_batch_samples_info, tsca_id, google_bucket_id)
    # Collect all samples
    all_samples = compile_samples(path_to_all_samples_info, google_bucket_id)
    # Add match normals and primaries
    batch_samples_with_matches = add_matching_samples(all_samples, batch_samples)
    write_df_to_csv(batch_samples_with_matches, tsca_id)
    return

In [54]:
def export_batch_metadata_to_fc(tsca_id):
    """Export metadata into Firecloud
    Args: 
        - tsca_id
    """
    patient_metadata    = "%s/fc_upload_patients_tsca_%s.txt" % (tsca_id, tsca_id)
    sample_set_metadata = "%s/fc_upload_sample_set_tsca_%s.txt" % (tsca_id, tsca_id)
    samples_metadata    = "%s/fc_upload_samples_tsca_%s.txt" % (tsca_id, tsca_id)
    pon_metadata        = "PoNs/fc_upload_PoN_sample_set_tsca_%s_PoN.txt" %(tsca_id)

    # Upload metadata
    r1 = upload_entities_from_tsv(namespace, workspace, patient_metadata)
    r2 = upload_entities_from_tsv(namespace, workspace, samples_metadata)
    r3 = upload_entities_from_tsv(namespace, workspace, sample_set_metadata)
    # r4 = upload_entities_from_tsv(namespace, workspace, pon_metadata)
    return (r1, r2, r3)

In [55]:
def create_cumulative_pon(paths, num_normals, pon_name, export_to_fc=False):
    """Create Cumulative PoN
    Args:
        - paths: list of paths to batch info files
        - tsca_ids: list of tsca_ids used in this batch
        - export_to_fc: export to firecloud
    """
    prepare_panel_of_normals_for_metadata_import(paths, num_normals, pon_name)
    if export_to_fc:
        return upload_entities_from_tsv(namespace, workspace, 'PoNs/fc_upload_PoN_%s.txt'%pon_name)
    return {}

In [56]:
def get_method_config_version(namespace, workspace, method_namespace, method_config_name):
    res = firecloud_api.get_workspace_config(namespace, workspace, method_namespace, method_config_name)
    return res.json()['methodRepoMethod']['methodVersion']

In [57]:
def update_method_config_versions():
    """Updates the file method_configs/latest_method_configs.txt
    This file contains the latest snapshot versions of the methods used in Firecloud
    """
    method_configs = pd.read_table('method_configs/latest_method_configs.txt')
    method_configs['snapshot'] = method_configs['method'].apply(lambda x: get_method_config_version(namespace, workspace, 'tsca', x))
    timestamp = datetime.datetime.now().strftime("%Y_%m_%d_%H:%M")
    method_configs.to_csv('method_configs/%s_method_configs.txt'%timestamp, index=False, sep="\t")
    method_configs.to_csv('method_configs/latest_method_configs.txt', index=False, sep="\t")
    return method_configs

In [63]:
def download_remote_wdls():
    """Update WDL scripts in wdls/production directory, to the ones currently being used in Firecloud
    """
    update_method_config_versions()
    method_configs = pd.read_table('method_configs/latest_method_configs.txt')
    for idx, method in method_configs.iterrows():
        res = firecloud_api.get_repository_method('tsca', method.method, method.snapshot)
        print("Updating WDL for %s:%s"%(method.method, method.snapshot))
        if res.status_code == 200:
            text_file = open("../wdls/production/%s.wdl"%method.method, "w")
            text_file.write(res.json()['payload'])
            text_file.close()
    return

In [61]:
def update_batch_metadata(tsca_id, path_to_batch_samples_info, path_to_all_samples_info):
    """Upload to Firecloud all the metadata necessary to run TSCA pipeline on new batch
    Args: 
        - tsca_id: tsca_id of the batch to run TSCA pipeline on
        - path_to_batch_samples_info: ends in *.import_samples.txt
        - path_to_all_samples_info: .xlsx file with path_to_batch_samples_info for all batches
    """
    # Prepare all metadata for batch
    prepare_batch_data_for_metadata_import(tsca_id, path_to_batch_samples_info, path_to_all_samples_info)
    # Upload to Firecloud
    export_batch_metadata_to_fc(batch_id)
    return

In [24]:
def main():
    """Run for every new batch.
    It is also necessary to upload metadata for all previous batches, because this batch can contain 
    match normals for samples in previous batches.
    """
    print ("For new batch: please add batch_id and path_to_batch_info to paths_to_samples_info.xlsx file")
    s = input('Have you updated the paths_to_samples_info.xlsx file? (Y/N)')
    if s == "N":
        print("Please do so before proceeding...")
        return
    path_to_all_samples_info = "paths_to_samples_info.xlsx"
    batches_info = pd.read_excel(path_to_all_samples_info)
#     for idx, batch in batches_info.iterrows():
#         update_batch_metadata(batch.tscaid, batch.path_to_samples_info, path_to_all_samples_info)

In [25]:
# main()

#### Common calls

In [22]:
### Prepare all metadata for batch
# prepare_all_metadata('TSCA20', '/xchip/clf/seq_data/processed_for_fh/tsca20_201707_SN0125362/tsca20_201707_SN0125362.import_samples.txt')

In [95]:
# attrs = ["__exac_vcf_index"]
# res = delete_entity_attributes(namespace, workspace, "sample", "AA66-Tumor-SM-F29RQ", attrs)

In [23]:
### Create PoN for all batches
# for index, value in paths_to_samples_info.iterrows():
#     paths = [value.path_to_samples_info]
#     pon_id = "%s_PoN" % value.tsca_id
#     panel_of_normals_for_metadata_import(paths, -1, pon_id)

In [40]:
### Export metadata for batch
# r1, r2, r3 = export_metadata('TSCA14')
# for index, value in paths_to_samples_info.iterrows():
#     export_metadata(value.tsca_id)

In [22]:
### Delete sample sets
# for tsca_id in paths_to_samples_info['tsca_id'].tolist():
#     delete_sample_set(namespace, workspace, "%s_PoN" %tsca_id)
# delete_sample_set(namespace, workspace, "CumPon40")

In [29]:
### Deleting a sample
# res = delete_sample_set(namespace, workspace, "TSCA14")
# res = delete_sample(namespace, workspace, 'DW039-Tumor-SM-DB2IF')
### NOTE: Delete sample (manually) from samples fc_upload file AND from sample_set membership fc_upload file

In [ ]:
### Create cumulative PoN
# res = create_cumulative_pon(batches_info.path_to_samples_info.tolist(), 5, 'CumPon5', True)

In [31]:
# all_samples = compile_samples('paths_to_samples_info.xlsx', google_bucket_id)

In [37]:
# pon = pd.read_table('PoNs/fc_upload_PoN_sample_set_tsca_CumPon40.txt')

In [100]:
# pon.merge(all_samples, left_on='sample_id', right_on='entity:sample_id')[['sample_id', 'tsca_id']]

In [28]:
# update_method_config_versions()

In [26]:
def download_remote_samples_metadata(filename):
    """Download metedata of all samples to filename
    Args:
        - filename: file to write samples metadata to
    """
    res = firecloud_api.get_entities_tsv(namespace, workspace, "sample")
    with open(filename, 'wb') as f:
        for chunk in res.iter_content(chunk_size=1024): 
            if chunk:
                f.write(chunk)
    return

In [27]:
def add_match_normal_oncotated_maf(target, all_samples):
    """For every sample, add the path to the match normal oncotated maf file if it exists
    Args:
        - target: sample we are looking the match normal for
        - all_samples: all samples to look for match normal in
    """
    if pd.isnull(target.match_normal_sample_id):
        return target
    match_normal = all_samples[ all_samples['entity:sample_id'] == target.match_normal_sample_id]
    if pd.notnull(match_normal.oncotated_maf.item()):
        print("Sample %s has normal with maf"%target['entity:sample_id'])
        target['match_normal_oncotated_maf'] = match_normal.oncotated_maf.item()
    return target

In [28]:
def update_samples_with_oncotated_mafs():
    """Read remote samples downloaded, and update every sample with the oncotated maf of match normal 
    if it exists.
    Return:
        - result
    """
    filename = "match_normal_updates/remote_samples.tsv"
    download_remote_samples_metadata(filename)
    remote_samples = pd.read_table(filename)
    updated_samples = remote_samples.apply(lambda row: add_match_normal_oncotated_maf(row, remote_samples), axis=1)
    updated_samples_select = updated_samples[['entity:sample_id', 'participant', 'match_normal_oncotated_maf']]
    updated_samples_select.to_csv("match_normal_updates/remote_samples_updated.tsv", sep="\t", index=None)
    res = upload_entities_from_tsv(namespace, workspace, "match_normal_updates/remote_samples_updated.tsv")
    return res

In [29]:
update_samples_with_oncotated_mafs()

Sample AA44-Normal-SM-DHZ88 has normal with maf
Sample CCLF_PEDS1019-Normal-SM-E7S24 has normal with maf
Sample CCLF_PEDS1019-Tumor-SM-E7S2G has normal with maf
Sample CCLF_PEDS1023-Tumor-SM-F29S7 has normal with maf
Sample CCLF_PEDS1027-Tumor-SM-EUVFW has normal with maf
Sample CCLF_PEDS1029-Normal-SM-EUVH1 has normal with maf
Sample CCLF_PEDS1029-Tumor-SM-EUVFZ has normal with maf
Sample CCLF_PEDS1033-Normal-SM-F29SD has normal with maf
Sample CCLF_PEDS1033-Tumor-SM-F29SE has normal with maf
Sample CCLF_PEDS1034-Tumor-SM-F67EF has normal with maf
Sample CCLF_PEDS1037-Tumor-SM-F67FK has normal with maf
Sample CCLF_RCRF1009-Tumor-SM-F67FI has normal with maf
Sample PEDS063-Tumor-SM-F67EL has normal with maf
Sample PEDS143-Tumor-SM-F67FL has normal with maf


<Response [200]>

In [109]:
def add_match_samples_from_current_batch(tsca_id, path_to_all_samples_info):
    """Add match primaries and normals in current batch for samples in previous batches
    Args:
        - tsca_id: current batch id
    """
    - read all samples except current one
    - read samples in current batch
    - for sample in all samples except current one:
    - look for primary and normal in current ones
    

In [168]:
all_batches_info      = pd.read_excel(path_to_all_samples_info)
previous_batches_info = all_batches_info.iloc[:-1, :]
current_batch_info    = all_batches_info.iloc[-1, :]

previous_samples = compile_samples(path_to_all_samples_info, google_bucket_id, previous_batches_info.tsca_id.tolist())
current_samples  = pd.read_table(current_batch_info.path_to_samples_info)

In [178]:
current_samples.columns

Index(['sample_id', 'individual_id', 'clean_bam_file_capture',
       'external_id_validation', 'aggregation_product_name_validation',
       'bsp_sample_id_validation', 'stock_sample_id_validation', 'sample_type',
       'picard_aggregation_type_validation', 'processed_subtype_validation',
       'source_subtype_validation', 'squid_sample_id_validation',
       'tumor_subtype', 'short_letter_code'],
      dtype='object')

In [187]:
samples_with_new_match_normal = []
samples_with_new_match_primary= []

for idx, row in previous_samples.iterrows():
    #print(row['entity:sample_id'])
    # Find all samples from same individual (same individual_id, different sample_id)
    patient_samples = current_samples[ (current_samples['individual_id'] == row['participant_id']) \
                                      & (current_samples['sample_id'] != row['entity:sample_id']) ]

    # NOTE: If more than one match tumor tissue or match normal found, select one at random.
    # Tumor tissue: Add primary tumor tissue
    match_primary_tumor = current_samples[ current_samples['external_id_validation'] \
                                          .str.contains('primary|prim|tissue|tiss') ]
    # Add match primary
    if match_primary_tumor.shape[0] > 0:
        print ("Sample %s in tsca %s has match primary in current batch" %(row['entity:sample_id'], row['tsca_id']))
        match_primary_tumor = match_primary_tumor.sample(n=1)
        samples_with_new_match_primary.append(match_primary_tumor.sample_id.item())

    # Add match normal
    match_normal = patient_samples[ patient_samples['sample_type'] == "Normal"]
    if match_normal.shape[0] > 0:
        print("Sample %s in tsca %s has match normal in current batch" % (row['entity:sample_id'], row['tsca_id']))
        match_normal = match_normal.sample(n=1)
        samples_with_new_match_normal.append(match_normal.sample_id.item())

Sample AA44-Normal-SM-DHZ88 in tsca TSCA15 has match normal in current batch
Sample CCLF_PEDS1019-Normal-SM-E7S24 in tsca TSCA17 has match normal in current batch
Sample CCLF_PEDS1019-Tumor-SM-E7S2G in tsca TSCA17 has match normal in current batch
Sample CCLF_PEDS1019-Normal-SM-EUVGI in tsca TSCA18 has match normal in current batch
Sample CCLF_PEDS1027-Tumor-SM-EUVFW in tsca TSCA18 has match normal in current batch
Sample CCLF_PEDS1029-Normal-SM-EUVH1 in tsca TSCA18 has match normal in current batch
Sample CCLF_PEDS1029-Tumor-SM-EUVFZ in tsca TSCA18 has match normal in current batch
Sample CCLF_PEDS1033-Normal-SM-F29SD in tsca TSCA19 has match normal in current batch
Sample CCLF_PEDS1033-Normal-SM-F29SC in tsca TSCA19 has match normal in current batch
Sample CCLF_PEDS1033-Tumor-SM-F29SF in tsca TSCA19 has match normal in current batch
Sample CCLF_PEDS1033-Tumor-SM-F29SE in tsca TSCA19 has match normal in current batch


In [77]:
for i in range(50, 60):
    firecloud_api.delete_repository_method('tsca', 'Mutect2TumorOnly', i)

In [ ]:
def add_matching_samples(all_samples, batch_samples):
    """Add sample_id and bam filepath of matching normals and primary tumor tissue for every sample
    in batch_samples
    Args:
        - all_samples: df with target samples we want to find matches in
        - batch_samples: df with source samples we want to find matches for
    Returns: 
        - batch_samples (augmented)
    """
    for index, row in batch_samples.iterrows():
        # Find all samples from same individual (same individual_id, different sample_id)
        patient_samples = all_samples[ (all_samples['participant_id'] == row['participant_id']) \
                                      & (all_samples['entity:sample_id'] != row['entity:sample_id']) ]

        # NOTE: If more than one match tumor tissue or match normal found, select one at random.
        # The match normal is used to compute allelic fractions in Mutect2, so for now we ignore the conditions it was grown in.

        # Tumor tissue: Add primary tumor tissue
        match_primary_tumor = patient_samples[ patient_samples['external_id_validation'] \
                                              .str.contains('primary|prim|tissue|tiss') ]
        #    > No primary tumor tissue found
        if match_primary_tumor.empty:
            batch_samples.loc[index, 'match_primary_tumor_sample_id'] = "NA"
            batch_samples.loc[index, 'match_primary_tumor_bam_file'] = "NA"
        #    > Tumor tissue found
        elif match_primary_tumor.shape[0] > 0:
            match_primary_tumor = match_primary_tumor.sample(n=1)
            batch_samples.loc[index, 'match_primary_tumor_sample_id'] = match_primary_tumor['entity:sample_id'].item()
            batch_samples.loc[index, 'match_primary_tumor_bam_file'] = match_primary_tumor['clean_bam_file_capture'].item()

        # Add match normal
        match_normal = patient_samples[ patient_samples['sample_type'] == "Normal"]
        #   > No match normal found
        if match_normal.empty: 
            batch_samples.loc[index, 'match_normal_sample_id'] = "NA"
            batch_samples.loc[index, 'match_normal_bam_file'] = "NA"
        #   > Match normal found
        elif match_normal.shape[0] > 0:
            match_normal = match_normal.sample(n=1)
            batch_samples.loc[index, 'match_normal_sample_id'] = match_normal['entity:sample_id'].item()
            batch_samples.loc[index, 'match_normal_bam_file'] = match_normal['clean_bam_file_capture'].item()
            
    return batch_samples